In [ ]:
!pip install -r requirements.txt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters for the gaussian distributions
mean1 = [0, 0]
cov1 = [[1, 0], [0, 100]]  # diagonal covariance

mean2 = [10, 10]
cov2 = [[10, 0], [0, 20]]  # diagonal covariance

# Generate the datasets
mc1 = np.random.multivariate_normal(mean1, cov1, 50000)
mc2 = np.random.multivariate_normal(mean2, cov2, 50000)

# Plot the datasets
plt.scatter(mc1[:, 0], mc1[:, 1], s=1)
plt.scatter(mc2[:, 0], mc2[:, 1], s=1)

plt.show()

In [ ]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Create the labels
mc1_labels = np.zeros(len(mc1))
mc2_labels = np.ones(len(mc2))

# Merge the datasets
X = np.concatenate((mc1, mc2))
y = np.concatenate((mc1_labels, mc2_labels))

# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

params = {
    "max_depth": [1, 2, 3, 4, 5, 6, 7],
    "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],
    "n_estimators": [10, 50, 100, 200, 300, 500],
}

# Create the model
model = XGBClassifier()

# Create the grid search object
grid = GridSearchCV(model, params, scoring="accuracy", n_jobs=-1, cv=5)

# Fit the grid search
grid.fit(X_train, y_train)

# Print the best parameters
print(grid.best_params_)

# Print the best score
print(grid.best_score_)

# Get best estimator
model = grid.best_estimator_

# Score the model
y_pred = model.predict(X_test)
predictions = [value for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# Plot the probability density functions of the classifier
y_pred_1 = model.predict_proba(X[y == 0])
y_pred_2 = model.predict_proba(X[y == 1])
n_bins = 100
plt.hist(
    y_pred_1[:, 1], bins=n_bins, alpha=0.5, label="mc1", log=True, density=True
)
plt.hist(
    y_pred_2[:, 1], bins=n_bins, alpha=0.5, label="mc2", log=True, density=True
)
plt.ylim(1e-2, 2e2)
plt.legend(loc="upper right")

In [ ]:
# Plot the probability density functions of the classifier
y_pred_1 = model.predict_proba(X_test[y_test == 0])
y_pred_2 = model.predict_proba(X_test[y_test == 1])
n_bins = 100
plt.hist(
    y_pred_1[:, 1], bins=n_bins, alpha=0.5, label="mc1", log=True, density=True
)
plt.hist(
    y_pred_2[:, 1], bins=n_bins, alpha=0.5, label="mc2", log=True, density=True
)
plt.ylim(1e-2, 2e2)
plt.legend(loc="upper right")

In [ ]:
new_data1 = np.random.multivariate_normal(mean1, cov1, 90000000)
new_data2 = np.random.multivariate_normal(mean2, cov2, 90000000)

new_labels1 = np.zeros(len(new_data1))
new_labels2 = np.ones(len(new_data2))

new_X = np.concatenate((new_data1, new_data2))
new_y = np.concatenate((new_labels1, new_labels2))

new_y_pred = model.predict_proba(new_X)[:, 1]
new_y_pred_1 = model.predict_proba(new_data1)[:, 1]
new_y_pred_2 = model.predict_proba(new_data2)[:, 1]

In [ ]:
# Set the y axis limits
plt.hist(
    new_y_pred_1, bins=n_bins, alpha=0.5, label="mc1", log=True, density=True
)
plt.hist(
    new_y_pred_2, bins=n_bins, alpha=0.5, label="mc2", log=True, density=True
)
plt.ylim(1e-3, 2e2)
plt.legend(loc="upper right")

In [ ]:
# Set the y axis limits
n_bins = int(1 + 3.3 * np.log(len(new_y_pred_1)))  # sturge's rule
plt.hist(
    new_y_pred_1, bins=n_bins, alpha=0.5, label="mc1", log=True, density=True
)
plt.hist(
    new_y_pred_2, bins=n_bins, alpha=0.5, label="mc2", log=True, density=True
)
plt.ylim(1e-3, 2e2)
plt.xlim(0, 1)
plt.legend(loc="upper right")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

# Fit a normal distribution to the data
params = stats.norm.fit(y_pred_1[:, 1])

# Plot the histogram
plt.hist(y_pred_1[:, 1], bins=61, density=True, alpha=0.6, color="g", log=True)

# Plot the PDF
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, *params)
plt.plot(x, p, "k", linewidth=2)

plt.ylim(1e-3, 2e2)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

# Fit a Weibull distribution to the data
params = stats.weibull_min.fit([x for x in y_pred_1[:, 1] if x < 0.5])
plt.figure(figsize=(6, 4))
# Plot the histogram
plt.hist(y_pred_1[:, 1], bins=61, density=True, alpha=0.6, color="g", log=True)

# Plot the PDF
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.weibull_min.pdf(x, *params)
plt.plot(x, p, "k", linewidth=2)

plt.ylim(1e-3, 2e2)
plt.show()

In [ ]:
# Draw the CDF of the data
plt.figure(figsize=(6, 4))
plt.hist(
    y_pred_1[:, 1],
    bins=61,
    density=True,
    histtype="step",
    cumulative=True,
    label="CDF",
)
# Add the cdf of the fitted distribution
plt.plot(
    x, p.cumsum() / p.cumsum().max(), "k--", linewidth=2, label="CDF fitted"
)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

# Fit an Inverse Gamma distribution to the data
params = stats.invgamma.fit([x for x in y_pred_1[:, 1] if x < 0.5])
plt.figure(figsize=(6, 4))

# Plot the histogram
plt.hist(y_pred_1[:, 1], bins=61, density=True, alpha=0.6, color="g", log=True)

# Plot the PDF
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.invgamma.pdf(x, *params)
plt.plot(x, p, "k", linewidth=2)

plt.ylim(1e-3, 2e2)
plt.show()

In [ ]:
# Draw the CDF of the data
plt.figure(figsize=(6, 4))
plt.hist(
    y_pred_1[:, 1],
    bins=61,
    density=True,
    histtype="step",
    cumulative=True,
    label="CDF",
)
# Add the cdf of the fitted distribution
plt.plot(
    x, p.cumsum() / p.cumsum().max(), "k--", linewidth=2, label="CDF fitted"
)
plt.show()

In [ ]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from scipy.stats._continuous_distns import _distn_names
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')

# Create models from data
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Best holders
    best_distributions = []

    # Estimate distribution parameters from data
    for ii, distribution in enumerate([d for d in _distn_names if not d in ['levy_stable', 'studentized_range']]):

        print("{:>3} / {:<3}: {}".format( ii+1, len(_distn_names), distribution ))

        distribution = getattr(st, distribution)

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')
                
                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]
                
                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))
                
                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                best_distributions.append((distribution, params, sse))
        
        except Exception:
            pass

    
    return sorted(best_distributions, key=lambda x:x[2])



In [ ]:
def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function"""

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = (
        dist.ppf(0.000001, *arg, loc=loc, scale=scale)
        if arg
        else dist.ppf(0.00001, loc=loc, scale=scale)
    )
    end = (
        dist.ppf(0.99999999, *arg, loc=loc, scale=scale)
        if arg
        else dist.ppf(0.9999999, loc=loc, scale=scale)
    )

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

In [ ]:
# Load data from statsmodels datasets
data = pd.DataFrame(y_pred_1[:, 1])

# Plot for comparison
plt.figure(figsize=(3, 2))
ax = data.plot(
    kind="hist",
    bins=50,
    density=True,
    alpha=0.5,
    color=list(matplotlib.rcParams["axes.prop_cycle"])[1]["color"],
    log=True,
)
# Save plot limits
dataYLim = ax.get_ylim()
# Find best fit distribution
best_distibutions = best_fit_distribution(data, 200, ax)
best_dist = best_distibutions[0]

# Update plots
ax.set_ylim(dataYLim)
ax.set_title("All Fitted Distributions")
ax.set_xlabel("Temp (°C)")
ax.set_ylabel("Frequency")

In [ ]:
# Make PDF with best params
pdf = make_pdf(best_dist[0], best_dist[1])

# Display
plt.figure(figsize=(6, 4))
ax = pdf.plot(lw=2, label="PDF", legend=True)
data.plot(
    kind="hist",
    bins=50,
    density=True,
    alpha=0.5,
    label="Data",
    legend=True,
    ax=ax,
    log=True,
)

param_names = (
    (best_dist[0].shapes + ", loc, scale").split(", ")
    if best_dist[0].shapes
    else ["loc", "scale"]
)
param_str = ", ".join(
    ["{}={:0.2f}".format(k, v) for k, v in zip(param_names, best_dist[1])]
)
dist_str = "{}({})".format(best_dist[0].name, param_str)

ax.set_title("best fit distribution \n" + dist_str)
ax.set_xlabel("Temp. (°C)")
ax.set_ylabel("Frequency")